<a href="https://colab.research.google.com/github/Akk216/GeNWrK/blob/main/EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch
import torch.nn as nn
from math import ceil

base_model = [
    # expand_ratio, channels, repeats, stride, kernel_size
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

phi_values = {
    # tuple of: (phi_value, resolution, drop_rate)
    "b0": (0, 224, 0.2),  # alpha, beta, gamma, depth = alpha ** phi
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

class CNNBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernelsize, stride, padding, groups=1):
    super(CNNBlock, self).__init__()
    self.cnn = nn.Conv2d(in_channels, out_channels, kernelsize, stride, padding, groups=groups, bias=False,)
    self.bn = nn.BatchNorm2d(out_channels)
    self.silu = nn.SiLU() # SiLU <-> Swish

  def forward(self, x):
      return self.silu(self.bn(self.cnn(x)))




class SqExc(nn.Module):
  def __init__(self, in_channels, reduced_dim):
    super(SqExc, self).__init__()
    self.se=nn.Sequential(
        nn.AdaptiveAvgPool2d(1),
        nn.Conv2d(in_channels, reduced_dim, 1),
        nn.SiLU(),
        nn.Conv2d(reduced_dim, in_channels, 1),
        nn.Sigmoid()     
    )
  def forward(self, x):
    return x*self.se(x)

class InvResBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernelsize, stride,  padding, ex_ratio, reduction=4, surv_prob=0.8):
    super(InvResBlock, self).__init__()
    self.surv_prob= surv_prob
    self.use_residual= in_channels == out_channels and stride == 1
    hdim=in_channels*ex_ratio
    self.expand = in_channels != hdim
    reddim=int(in_channels / reduction)

    if self.expand:
      self.expand_conv = CNNBlock( in_channels, hdim, kernelsize=3, stride= 1, padding=1)
    
    self.conv= nn.Sequential(
        CNNBlock(hdim, hdim, kernelsize, stride, padding, groups= hdim),
        SqExc(hdim, reddim),
        nn.Conv2d(hdim, out_channels, 1, bias=False),
        nn.BatchNorm2d(out_channels),
    )

  def stochastic_depth(self, x):
    if not self.training:
      return x

    binary_tensor = torch.rand(x.shape[0],1,1,1,device=x.device) < self.surv_prob 
    return torch.div(x, self.surv_prob)* binary_tensor

  def forward(self, inputs):
    x=self.expand_conv(inputs) if self.expand else inputs

    if self.use_residual:
      return self.stochastic_depth(self.conv(x)) + inputs
    else:
      return self.conv(x)  
         
class EfficientNet(nn.Module):
  def __init__(self, version, num_classes):
    super(EfficientNet, self).__init__()
    wf,df,dropout_rate=self.cal(version)
    last_channel= ceil(1280 * wf)
    self.pool = nn.AdaptiveAvgPool2d(1)
    self.features=self.create_features(wf, df, last_channel)
    self.classifier=nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(last_channel, num_classes),
    )
  
  def cal(self, version, alpha=1.2, beta=1.1):
    phi, res, droprate= phi_values[version]
    df= alpha ** phi
    wf = beta ** phi
    return wf,df,droprate
  
  def create_features(self, wf, df, last_channel):
    channels=int(32*wf)
    features = [CNNBlock(3, channels, 3, stride =2, padding= 1)]
    in_channels = channels

    for expand_ratio, channels, repeats, stride, kernelsize in base_model:
      out_channels = 4*ceil(int(channels*wf)/4)
      layer_repeats = ceil(repeats * df)

      for layer in range(layer_repeats):
        features.append(InvResBlock(in_channels, out_channels, ex_ratio= expand_ratio, stride= stride if layer ==0 else 1, kernelsize=kernelsize, padding = kernelsize//2,))
        in_channels=out_channels
    features.append(
        CNNBlock(in_channels, last_channel, kernelsize=1, stride = 1, padding = 0)
    )
    return nn.Sequential(*features)
  
  def forward(self, x):
    x = self.pool(self.features(x))
    return self.classifier(x.view(x.shape[0],-1))



In [21]:
def test():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    version = "b0"
    phi, res, drop_rate = phi_values[version]
    num_examples, num_classes = 4, 10
    x = torch.randn((num_examples, 3, res, res)).to(device)
    model = EfficientNet(
        version=version,
        num_classes=num_classes,
    ).to(device)

    print(model(x).shape) 



In [22]:
test()

torch.Size([4, 10])
